## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 100].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
Other,669
C1700,287
C4000,194


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "CLASSIFICATION"],drop_first=True)


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [ ]:
# Create a StandardScaler instances

X_train = pd.DataFrame(X_train, columns=application_df.drop(columns=["IS_SUCCESSFUL"]).columns)
X_test = pd.DataFrame(X_test, columns=application_df.drop(columns=["IS_SUCCESSFUL"]).columns)

categorical_cols = X_train.select_dtypes(include=['object']).columns
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)



X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


scaler = StandardScaler()
# Fit the StandardScaler

X_scaler = scaler.fit(X_train)


# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [ ]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
import tensorflow as tf
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_shape=(X_train_scaled.shape[1],)))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │       531,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533,581 (2.04 MB)

 Trainable params: 533,581 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6628 - loss: 0.6456 - val_accuracy: 0.6974 - val_loss: 0.5873
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7965 - loss: 0.4522 - val_accuracy: 0.6762 - val_loss: 0.8633
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8080 - loss: 0.4063 - val_accuracy: 0.6797 - val_loss: 0.7759
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8106 - loss: 0.3988 - val_accuracy: 0.6762 - val_loss: 0.7916
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8110 - loss: 0.3979 - val_accuracy: 0.6772 - val_loss: 0.8532
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8119 - loss: 0.3977 - val_accuracy: 0.6754 - val_loss: 0.8575
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8163 - loss: 0.3896 - val_accuracy: 0.6814 - val_loss: 0.7583
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.8143 - loss: 0.3922 - val_accuracy: 0.

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7273 - loss: 0.6433
Loss: 0.643287718296051, Accuracy: 0.7273469567298889


In [16]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2)
loss1, acc1 = model1.evaluate(X_test_scaled, y_test)
print(f"Attempt 1 Accuracy: {acc1:.4f}")

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6549 - loss: 0.6626 - val_accuracy: 0.7005 - val_loss: 0.5802
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.8032 - loss: 0.4265 - val_accuracy: 0.6993 - val_loss: 0.6268
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8054 - loss: 0.4064 - val_accuracy: 0.7046 - val_loss: 0.6038
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8091 - loss: 0.4004 - val_accuracy: 0.7024 - val_loss: 0.6127
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8121 - loss: 0.3996 - val_accuracy: 0.7048 - val_loss: 0.6073
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8079 - loss: 0.4019 - val_accuracy: 0.6970 - val_loss: 0.6434
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8081 - loss: 0.4014 - val_accuracy: 0.6805 - val_loss: 0.7591
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8105 - loss: 0.3989 - va

In [18]:
from tensorflow.keras.layers import Dropout

model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2)
loss2, acc2 = model2.evaluate(X_test_scaled, y_test)
print(f"Attempt 2 Accuracy: {acc2:.4f}")

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6263 - loss: 0.7516 - val_accuracy: 0.6964 - val_loss: 0.5768
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7422 - loss: 0.5493 - val_accuracy: 0.7044 - val_loss: 0.5806
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7978 - loss: 0.4396 - val_accuracy: 0.6974 - val_loss: 0.6323
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8080 - loss: 0.4148 - val_accuracy: 0.6941 - val_loss: 0.7181
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8082 - loss: 0.4134 - val_accuracy: 0.6980 - val_loss: 0.6407
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8084 - loss: 0.4073 - val_accuracy: 0.6927 - val_loss: 0.7578
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8083 - loss: 0.4080 - val_accuracy: 0.6941 - val_loss: 0.6406
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.8125 - loss: 0.4021 - val_accuracy

In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.keras")


In [19]:
model2.save("AlphabetSoupCharity_Optimization.h5")